In [1]:
from tile_classifier import TileClassifier

from keras import backend as K
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

def export_model(saver, model, input_node_names, output_node_name,MODEL_NAME):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

Using TensorFlow backend.
/home/piotr/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
if __name__ == '__main__':
    model_path ='tile_classifier.h5'
    data_set_path = './data/data_set_tile'
    model = TileClassifier(data_set_path,model_path)
    model.train_evaluate(epochs=5)
    class_indices = model.get_class_indices()
    print(class_indices)
    

Found 1868 images belonging to 2 classes.
Epoch 1/5
1494/1494 [==============================] - 29s 20ms/step - loss: 0.4268 - acc: 0.8333
Epoch 2/5
1494/1494 [==============================] - 29s 20ms/step - loss: 0.0808 - acc: 0.9692
Epoch 3/5
1494/1494 [==============================] - 19s 13ms/step - loss: 0.0310 - acc: 0.9920
Epoch 4/5
1494/1494 [==============================] - 20s 13ms/step - loss: 0.0070 - acc: 0.9987
Epoch 5/5
374/374 [==============================] - 2s 4ms/step
[0.0008093708401087069, 1.0]
{'non-tiles': 0, 'tiles': 1}


In [5]:
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax", "tile_classifier")

INFO:tensorflow:Restoring parameters from out/tile_classifier.chkp
INFO:tensorflow:Froze 12 variables.
Converted 12 variables to const ops.
graph saved!


In [6]:
model

In [12]:
model.classifier.input_shape

(None, 64, 64, 3)

In [15]:
150528 / 3

50176.0

In [16]:
import numpy as np

In [17]:
np.sqrt(50176)

224.0